In [ ]:
import redis
import json

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)

# Khóa của tập hợp
redis_set_key = "unique_get_picker_ids"

# Lấy tất cả các phần tử từ tập hợp
set_values = redis_client.smembers(redis_set_key)

# In ra các phần tử và giá trị tương ứng
for value in set_values:
    decoded_value = value.decode('utf-8')
    print(f"Set member: {decoded_value}")

    # Lấy giá trị của mỗi khóa tương ứng
    redis_key = f"match:{decoded_value}"
    match_value = redis_client.get(redis_key)
    
    if match_value:
        match_value = json.loads(match_value.decode('utf-8'))
        print(f"Match data for {redis_key}: {match_value}")
    else:
        print(f"No data found for {redis_key}")


In [ ]:
import redis
import json

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)

# Khóa của tập hợp
redis_set_key = "unique_get_picker_ids"

# Lấy tất cả các phần tử từ tập hợp
set_values = redis_client.smembers(redis_set_key)

# Xóa các phần tử và các khóa tương ứng
for value in set_values:
    decoded_value = value.decode('utf-8')
    print(f"Deleting set member: {decoded_value}")

    # Xóa khóa tương ứng với mỗi phần tử
    redis_key = f"match:{decoded_value}"
    redis_client.delete(redis_key)
    print(f"Deleted key: {redis_key}")

# Xóa chính tập hợp
redis_client.delete(redis_set_key)
print(f"Deleted set: {redis_set_key}")


In [ ]:
import redis
import json

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)

# Khóa của tập hợp
redis_set_key = "unique_get_picker_ids"
def delete_redis_key(key_pattern):
    keys = redis_client.keys(key_pattern)
    if keys:
        redis_client.delete(*keys)
        print(f"Deleted keys: {keys}")
    else:
        print("No keys found to delete.")

delete_redis_key("Picker*")  # Xóa các key Redis trước khi bắt đầu


In [ ]:
import redis
import json

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)

# Khóa của tập hợp
redis_set_key = "unique_get_picker_ids"

# Lấy tất cả các phần tử từ tập hợp
set_values = redis_client.smembers(redis_set_key)

# Xóa các phần tử và các khóa tương ứng
for value in set_values:
    decoded_value = value.decode('utf-8')
    print(f"Deleting set member: {decoded_value}")

    # Xóa khóa tương ứng với mỗi phần tử
    redis_key = f"Picker{decoded_value}"
    redis_client.delete(redis_key)
    print(f"Deleted key: {redis_key}")

# Xóa chính tập hợp
redis_client.delete(redis_set_key)
print(f"Deleted set: {redis_set_key}")


In [ ]:
def delete_catche_redis(redis_set_key):
    # Kết nối đến Redis
    redis_client = redis.Redis(host='localhost', port=6377, db=0)

    # Lấy tất cả các phần tử từ tập hợp
    set_values = redis_client.smembers(redis_set_key)

    # Xóa các phần tử và các khóa tương ứng
    for value in set_values:
        decoded_value = value.decode('utf-8')
        print(f"Deleting set member: {decoded_value}")

        # Xóa khóa tương ứng với mỗi phần tử
        redis_key = f"match:{decoded_value}"
        redis_client.delete(redis_key)
        print(f"Deleted key: {redis_key}")

    # Xóa chính tập hợp
    redis_client.delete(redis_set_key)
    print(f"Deleted set: {redis_set_key}")

In [ ]:
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
from config.teams import CODE2TEAMNAME
import pandas as pd
from Utils.postgres_tool import PostgresTool
from datetime import datetime
import pytz
import requests
import redis 
import emoji
import yml

config = yml.read_config("./config/config.yml")

# headers = read_headers_from_config("./config/config.ini")
# params = read_params_from_config("./config/config.ini")
# url_factmath = config['url']['URL_FACT_MATH']

headers = config['headers']
params = config['params']
url_factmath = config['url']['URL_FACT_MATH']
db_config = {
    "host": config['database']['DB_SV_HOSTS'],
    "port": config['database'].getint('DB_SV_PORT'),
    "user": config['database']['DB_SV_USER'],
    "password": config['database']['DB_SV_PASSWORD'],
    "database": config['database']['DB_SV_DATABASE']
}

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)
redis_set_key = "unique_match_ids"

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  
    return response.json()

def process_data(data, id_idx):
    for i in data:
        datas = {
            "id": i["id"],
            "home_team_id": i["home_team_id"],
            "away_team_id": i["road_team_id"],
            "status": i["game_state"],
            "home_team_score": i["home_team_score"],
            "away_team_score": i["road_team_score"],
            "live_home_team_score": i["live_home_team_score"],
            "live_away_team_score": i["live_road_team_score"],
            "home_team_spread": i["home_team_spread"],
            "away_team_spread": i["road_team_spread"],
            "point_total": i["over_under"],
            "date_id": id_idx,
            "kickoff_time": i["kickoff"].split('T')[0],
            "sport_id": "mlb",
        }
        yield datas

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def get_math_data(db_config, sport_id, season):
    conn = PostgresTool(**db_config)
    CURRENT_DATE_US = get_us_date()

    query = f"SELECT idx, id FROM date_id WHERE date_play = '{CURRENT_DATE_US}' AND sport_id = '{sport_id}' AND season = '2024'"
    idx = conn.query(query, False)[0][0]
    id_idx = conn.query(query, False)[0][1]
    querys1 = f"""SELECT COUNT(*) FROM "match" m WHERE m.date_id = '{id_idx-1}' AND m.status != 'Final'"""
    count = conn.query(querys1, False)[0][0]
    if count != 0:
        id_idx = id_idx - 1
        idx = idx - 1
    data = fetch_data_from_url(url_factmath.format(idx), headers, params)
    datas = process_data(data, id_idx)
    for i in datas:
        yield i
    conn.close()

def main(db_config):
    datas = get_math_data(db_config, "mlb", "2024")  # sport_id = "mlb", season = "2024"

    for data in datas:
        redis_key = f"match:{data}"
        exists = redis_client.sismember(redis_set_key, str(data))
        if not exists:
            # Lưu trữ dữ liệu vào Redis Set
            redis_client.sadd(redis_set_key, str(data))
            redis_client.expire(redis_set_key, 60*60*24)  # `60*60*24` seconds = 1 day
            redis_client.setex(redis_key, 60*60*24, str(data))
            # Chuyển đổi lại chuỗi đặc biệt thành None trước khi đẩy dữ liệu vào DB
            data = {k: (v if v != "None" else None) for k, v in data.items()}
            # Lưu trữ dữ liệu vào DB
            conn = PostgresTool(**db_config)
            conn.push_data('match', data)
            conn.close()
        else:
            print(emoji.emojize(f"🦄👩🏻‍💻 Duplicate data found, skipping: {data} 🦄"))

if __name__ == '__main__':
    main(db_config)


In [ ]:
import redis
import requests
import emoji
import configparser
from datetime import datetime
import pytz
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
from Utils.postgres_tool import PostgresTool

config = configparser.ConfigParser()
config.read('./config/config.ini')

headers = read_headers_from_config("./config/config.ini")
params = read_params_from_config("./config/config.ini")
url_factmath = config['url']['URL_FACT_MATH']

db_config = {
    "host": config['database']['DB_SV_HOSTS'],
    "port": config['database'].getint('DB_SV_PORT'),
    "user": config['database']['DB_SV_USER'],
    "password": config['database']['DB_SV_PASSWORD'],
    "database": config['database']['DB_SV_DATABASE']
}

db_config_sv = {
    'host': config['database_sv']['DB_SV_HOSTS'],
    'port': config['database_sv'].getint('DB_SV_PORT'),
    'user': config['database_sv']['DB_SV_USER'],
    'password': config['database_sv']['DB_SV_PASSWORD'],
    'database': config['database_sv']['DB_SV_DATABASE']
}

redis_client = redis.Redis(host='localhost', port=6377, db=0)
redis_set_key = "unique_match_ids"

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  
    return response.json()

def process_data(data, id_idx):
    for i in data:
        yield {
            "id": i["id"],
            "home_team_id": i["home_team_id"],
            "away_team_id": i["road_team_id"],
            "status": i["game_state"],
            "home_team_score": i["home_team_score"],
            "away_team_score": i["road_team_score"],
            "live_home_team_score": i["live_home_team_score"],
            "live_away_team_score": i["live_road_team_score"],
            "home_team_spread": i["home_team_spread"],
            "away_team_spread": i["road_team_spread"],
            "point_total": i["over_under"],
            "date_id": id_idx,
            "kickoff_time": i['additional_data']["DateTime"],
            "sport_id": "mlb",
        }

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def get_math_data(db_config, sport_id, season):
    conn = PostgresTool(**db_config)
    current_date_us = get_us_date()

    query = f"SELECT idx, id FROM date_id WHERE date_play = '{current_date_us}' AND sport_id = '{sport_id}' AND season = '2024'"
    idx, id_idx = conn.query(query, False)[0]

    querys1 = f"SELECT COUNT(*) FROM match WHERE date_id = '{id_idx-1}' AND status = 'Scheduled'"
    count = conn.query(querys1, False)[0][0]
    if count != 0:
        id_idx -= 1
        idx -= 1

    data = fetch_data_from_url(url_factmath.format(idx), headers, params)
    yield from process_data(data, id_idx)

    conn.close()

def main(db_config):
    datas = get_math_data(db_config, "mlb", "2024")

    for data in datas:
        redis_key = f"match:{data['id']}"  # Sử dụng id làm khóa duy nhất
        if not redis_client.sismember(redis_set_key, data['id']):
            redis_client.sadd(redis_set_key, data['id'])
            redis_client.expire(redis_set_key, 60*60*24)  # 1 day expiration
            redis_client.setex(redis_key, 60*60*24, str(data))
            data = {k: (v if v != "None" else None) for k, v in data.items()}
            conn = PostgresTool(**db_config)
            conn.push_data('match', data)
            conn.close()
        else:
            print(emoji.emojize(f"🦄👩🏻‍💻 Duplicate data found, skipping: {data} 🦄"))

if __name__ == '__main__':
    main(db_config)
    main(db_config_sv)


# Main backup



In [ ]:
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
from config.teams import CODE2TEAMNAME
import pandas as pd
import os
import configparser
from Utils.postgres_tool import PostgresTool
from datetime import datetime
import pytz
import requests
import redis 
import emoji
config = configparser.ConfigParser()
config.read('./config/config.ini')

headers = read_headers_from_config("./config/config.ini")
params = read_params_from_config("./config/config.ini")
url_factmath = config['url']['URL_FACT_MATH']
db_config = {
    "host": config['database']['DB_SV_HOSTS'],
    "port": config['database'].getint('DB_SV_PORT'),
    "user": config['database']['DB_SV_USER'],
    "password": config['database']['DB_SV_PASSWORD'],
    "database": config['database']['DB_SV_DATABASE']
}

db_config_sv = {
    'host': config['database_sv']['DB_SV_HOSTS'],
    'port': config['database_sv'].getint('DB_SV_PORT'),
    'user': config['database_sv']['DB_SV_USER'],
    'password': config['database_sv']['DB_SV_PASSWORD'],
    'database': config['database_sv']['DB_SV_DATABASE']
}
# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)
redis_set_key = "unique_match_ids"

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  
    return response.json()

def process_data(data, id_idx):
    for i in data:
        datas = {
            "id": i["id"],
            "home_team_id": i["home_team_id"],
            "away_team_id": i["road_team_id"],
            "status": i["game_state"],
            "home_team_score": i["home_team_score"],
            "away_team_score": i["road_team_score"],
            "live_home_team_score": i["live_home_team_score"],
            "live_away_team_score": i["live_road_team_score"],
            "home_team_spread": i["home_team_spread"],
            "away_team_spread": i["road_team_spread"],
            "point_total": i["over_under"],
            "date_id": id_idx,
            "kickoff_time": i['additional_data']["DateTime"],
            "sport_id": "mlb",
        }
        yield datas

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def get_math_data(db_config, sport_id, season):
    conn = PostgresTool(**db_config)
    CURRENT_DATE_US = get_us_date()

    query = f"SELECT idx, id FROM date_id WHERE date_play = '{CURRENT_DATE_US}' AND sport_id = '{sport_id}' AND season = '2024'"
    idx = conn.query(query, False)[0][0]
    id_idx = conn.query(query, False)[0][1]
    querys1 = f"""SELECT COUNT(*) FROM "match" m WHERE m.date_id = '{id_idx-1}' AND m.status = 'Scheduled'"""
    count = conn.query(querys1, False)[0][0]
    if count != 0:
        id_idx = id_idx - 1
        idx = idx - 1
    data = fetch_data_from_url(url_factmath.format(idx), headers, params)
    datas = process_data(data, id_idx)
    for i in datas:
        yield i
    conn.close()

def main(db_config):
    datas = get_math_data(db_config, "mlb", "2024")  # sport_id = "mlb", season = "2024"

    for data in datas:
        redis_key = f"match:{data}"
        exists = redis_client.sismember(redis_set_key, str(data))
        if not exists:
            # Lưu trữ dữ liệu vào Redis Set
            redis_client.sadd(redis_set_key, str(data))
            redis_client.expire(redis_set_key, 60*60*24)  # `60*60*24` seconds = 1 day
            redis_client.setex(redis_key, 60*60*24, str(data))
            # Chuyển đổi lại chuỗi đặc biệt thành None trước khi đẩy dữ liệu vào DB
            data = {k: (v if v != "None" else None) for k, v in data.items()}
            # Lưu trữ dữ liệu vào DB
            conn = PostgresTool(**db_config)
            conn.push_data('match', data)
            conn.close()
        else:
            print(emoji.emojize(f"🦄👩🏻‍💻 Duplicate data found, skipping: {data} 🦄"))

if __name__ == '__main__':
    main(db_config)
    main(db_config_sv)


# Main run

In [ ]:
import redis
import requests
import emoji
import configparser
from datetime import datetime
import pytz
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
from Utils.postgres_tool import PostgresTool

config = configparser.ConfigParser()
config.read('./config/config.ini')

headers = read_headers_from_config("./config/config.ini")
params = read_params_from_config("./config/config.ini")
url_factmath = config['url']['URL_FACT_MATH']

db_config = {
    "host": config['database']['DB_SV_HOSTS'],
    "port": config['database'].getint('DB_SV_PORT'),
    "user": config['database']['DB_SV_USER'],
    "password": config['database']['DB_SV_PASSWORD'],
    "database": config['database']['DB_SV_DATABASE']
}

db_config_sv = {
    'host': config['database_sv']['DB_SV_HOSTS'],
    'port': config['database_sv'].getint('DB_SV_PORT'),
    'user': config['database_sv']['DB_SV_USER'],
    'password': config['database_sv']['DB_SV_PASSWORD'],
    'database': config['database_sv']['DB_SV_DATABASE']
}

redis_client = redis.Redis(host='localhost', port=6377, db=0)

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  
    return response.json()

def process_data(data, id_idx):
    for i in data:
        yield {
            "id": i["id"],
            "home_team_id": i["home_team_id"],
            "away_team_id": i["road_team_id"],
            "status": i["game_state"],
            "home_team_score": i["home_team_score"],
            "away_team_score": i["road_team_score"],
            "live_home_team_score": i["live_home_team_score"],
            "live_away_team_score": i["live_road_team_score"],
            "home_team_spread": i["home_team_spread"],
            "away_team_spread": i["road_team_spread"],
            "point_total": i["over_under"],
            "date_id": id_idx,
            "kickoff_time": i['additional_data']["DateTime"],
            "sport_id": "mlb",
        }

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def get_math_data(db_config, sport_id, season):
    conn = PostgresTool(**db_config)
    current_date_us = get_us_date()

    query = f"SELECT idx, id FROM date_id WHERE date_play = '{current_date_us}' AND sport_id = '{sport_id}' AND season = '2024'"
    idx, id_idx = conn.query(query, False)[0]

    querys1 = f"SELECT COUNT(*) FROM match WHERE date_id = '{id_idx-1}' AND status = 'Scheduled'"
    count = conn.query(querys1, False)[0][0]
    if count != 0:
        id_idx -= 1
        idx -= 1

    data = fetch_data_from_url(url_factmath.format(idx), headers, params)
    yield from process_data(data, id_idx)

    conn.close()

def main(db_config, redis_prefix):
    redis_set_key = f"{redis_prefix}_unique_match_ids"
    datas = get_math_data(db_config, "mlb", "2024")

    for data in datas:
        redis_key = f"{redis_prefix}:match:{data}"  # Sử dụng id làm khóa duy nhất
        exists = redis_client.sismember(redis_set_key, str(data))

        if not exists:
            # Lưu trữ dữ liệu vào Redis Set
            redis_client.sadd(redis_set_key, str(data))
            redis_client.expire(redis_set_key, 60*60*24)  # `60*60*24` seconds = 1 day
            redis_client.setex(redis_key, 60*60*24, str(data))
            # Chuyển đổi lại chuỗi đặc biệt thành None trước khi đẩy dữ liệu vào DB
            data = {k: (v if v != "None" else None) for k, v in data.items()}
            # Lưu trữ dữ liệu vào DB
            conn = PostgresTool(**db_config)
            conn.push_data('match', data)
            conn.close()
        else:
            print(emoji.emojize(f"🦄👩🏻‍💻 {redis_prefix} Duplicate data found, skipping: {data} 🦄"))

if __name__ == '__main__':
    main(db_config, "db_config")
    main(db_config_sv, "db_config_sv")


# main fix push error

In [ ]:
import redis
import requests
import emoji
import configparser
from datetime import datetime
import pytz
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
from Utils.postgres_tool import PostgresTool

config = configparser.ConfigParser()
config.read('./config/config.ini')

headers = read_headers_from_config("./config/config.ini")
params = read_params_from_config("./config/config.ini")
url_factmath = config['url']['URL_FACT_MATH']

# config for database server bbsw
db_config = {
    "host": config['database']['DB_SV_HOSTS'],
    "port": config['database'].getint('DB_SV_PORT'),
    "user": config['database']['DB_SV_USER'],
    "password": config['database']['DB_SV_PASSWORD'],
    "database": config['database']['DB_SV_DATABASE']
}

# config for database server aws
db_config_sv = {
    'host': config['database_sv']['DB_SV_HOSTS'],
    'port': config['database_sv'].getint('DB_SV_PORT'),
    'user': config['database_sv']['DB_SV_USER'],
    'password': config['database_sv']['DB_SV_PASSWORD'],
    'database': config['database_sv']['DB_SV_DATABASE']
}

# connect to redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  
    return response.json()

def process_data(data, id_idx):
    for i in data:
        yield {
            "id": i["id"],
            "home_team_id": i["home_team_id"],
            "away_team_id": i["road_team_id"],
            "status": i["game_state"],
            "home_team_score": i["home_team_score"],
            "away_team_score": i["road_team_score"],
            "live_home_team_score": i["live_home_team_score"],
            "live_away_team_score": i["live_road_team_score"],
            "home_team_spread": i["home_team_spread"],
            "away_team_spread": i["road_team_spread"],
            "point_total": i["over_under"],
            "date_id": id_idx,
            "kickoff_time": i['additional_data']["DateTime"],
            "sport_id": "mlb",
        }

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def delete_catche_redis(redis_set_key):
    set_values = redis_client.smembers(redis_set_key)
    for value in set_values:
        decoded_value = value.decode('utf-8')
        redis_key = f"match:{decoded_value}"
        redis_client.delete(redis_key)

    redis_client.delete(redis_set_key)

def get_math_data(db_config, sport_id, season,redis_set_key):
    conn = PostgresTool(**db_config)
    current_date_us = get_us_date()

    query = f"SELECT idx, id FROM date_id WHERE date_play = '{current_date_us}' AND sport_id = '{sport_id}' AND season = '2024'"
    idx, id_idx = conn.query(query, False)[0]

    query_check_status = f"SELECT COUNT(*) FROM match WHERE date_id = '{id_idx-1}' AND status = 'Scheduled'"
    count = conn.query(query_check_status, False)[0][0]
    if count != 0:
        id_idx -= 1
        idx -= 1
    query_check_count = f"SELECT COUNT(*) FROM match WHERE date_id = '{id_idx}' "
    count_check = conn.query(query_check_count, False)[0][0]
    data = fetch_data_from_url(url_factmath.format(idx), headers, params)
    
    if count_check != 0 and count_check > len(data):

        query_delete = f"DELETE FROM match WHERE date_id = '{id_idx}'"
        conn.query(query_delete, False)
        # clear all data in redis
        delete_catche_redis(redis_set_key)
    conn.close()

    yield from process_data(data, id_idx)
 
def main(db_config, redis_prefix):
    redis_set_key = f"{redis_prefix}_unique_match_ids"
    datas = get_math_data(db_config, "mlb", "2024",redis_set_key)

    for data in datas:
        redis_key = f"{redis_prefix}:match:{data}"  # Sử dụng id làm khóa duy nhất
        exists = redis_client.sismember(redis_set_key, str(data))

        if not exists:
            # Lưu trữ dữ liệu vào Redis Set
            redis_client.sadd(redis_set_key, str(data))
            redis_client.expire(redis_set_key, 60*60*24)  # `60*60*24` seconds = 1 day
            redis_client.setex(redis_key, 60*60*24, str(data))
            # Chuyển đổi lại chuỗi đặc biệt thành None trước khi đẩy dữ liệu vào DB
            data = {k: (v if v != "None" else None) for k, v in data.items()}
            # Lưu trữ dữ liệu vào DB
            conn = PostgresTool(**db_config)
            conn.push_data('match', data)
            conn.close()
        else:
            print(emoji.emojize(f"🦄👩🏻‍💻 {redis_prefix} Duplicate data found, skipping: {data} 🦄"))

if __name__ == '__main__':
    main(db_config, "dbBBSW")
    # main(db_config_sv, "dbAWS")


# picker 

In [ ]:
from Utils.header import read_headers_from_config
from Utils.params import read_params_from_config
import configparser
from Utils.postgres_tool import PostgresTool
from datetime import datetime
import pytz
import requests
import redis

config = configparser.ConfigParser()
config.read('./config/config.ini')

headers = read_headers_from_config("./config/config.ini")
params = read_params_from_config("./config/config.ini")
url_picker = config['url']['URL_PICKER']
db_config = {
    "host": config['database_sv']['DB_SV_HOSTS'],
    "port": config['database_sv'].getint('DB_SV_PORT'),
    "user": config['database_sv']['DB_SV_USER'],
    "password": config['database_sv']['DB_SV_PASSWORD'],
    "database": config['database_sv']['DB_SV_DATABASE']
}

# Kết nối đến Redis
redis_client = redis.Redis(host='localhost', port=6377, db=0)
redis_set_key = "unique_get_picker_ids_test"

def fetch_data_from_url(url, headers, params):
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # Ensure we catch any HTTP errors
    return response.json()["picks"]

def process_pick_data(data, gameType):
    for i in data:
        for game, pick in i["picks"].items():
            datas = {
                "match_id": game,
                "date_add": datetime.now().strftime("%Y-%m-%d"),
                "status_update": True,
                "stake": pick.get("stake"),
                "seq": pick.get("seq"),
                "result": pick.get("result"),
                "win_pct": i.get("season_win_pct", 0)  # Default to 0 if season_win_pct is missing
            }
            if gameType == "ats":
                if "spread" in pick and "team_id" in pick and i.get("win_pct", 0) >= 55:
                    datas["spread"] = pick["spread"]
                    datas["team_id"] = pick["team_id"]
                    yield datas
                else:
                    continue
            else:
                if "ou_type" in pick and "ou_value" in pick and i.get("win_pct", 0) >= 51:
                    datas["ou_type"] = pick["ou_type"]
                    datas["ou_value"] = pick["ou_value"]
                    yield datas
                else:
                    continue

def delete_redis_key(key_pattern):
    keys = redis_client.keys(key_pattern)
    if keys:
        redis_client.delete(*keys)
        print(f"Deleted keys: {keys}")
    else:
        print("No keys found to delete.")

def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')

def get_picker(tournament, season, gameType, db_config,season_date):
    conn = PostgresTool(**db_config)
    CURRENT_DATE_US = get_us_date()

    query = f"SELECT idx FROM date_id WHERE date_play = '{CURRENT_DATE_US}' and sport_id = '{tournament}' and season = '{season_date}'"
    idx = conn.query(query, False)[0][0]

    list_url = [url_picker.format(tournament, season, idx, gameType, sequence) for sequence in range(0, 150, 50)]
    for url in list_url:
        data = fetch_data_from_url(url, headers, params)
        for datas in process_pick_data(data, gameType):
            redis_key = f"Picker{datas}"
            print(datas)
            # exists = redis_client.sismember(redis_set_key, str(datas))
            
            # if not exists:
            #     redis_client.sadd(redis_set_key,str(datas))
            #     redis_client.expire(redis_set_key, 60*60*24)
            #     redis_client.setex(redis_key, 60*60*24, str(datas))
            #     datas = {k: v for k, v in datas.items() if v is not None}
            #     if datas.get("win_pct") >= 55 and gameType == "ats" and datas.get("result") == "pregame":
            #         print([datas])
            #         # conn.push_data(f"expert_{gameType}",datas)
            #     # elif datas.get("win_pct") >= 51 and gameType == "ou" and datas.get("result") == "pregame":
            #         # conn.push_data(f"expert_{gameType}",datas)
            #     else:
            #         continue
            # else:
            #     print(f"🦄👩🏻‍💻 Duplicate data found, skipping: {datas}")
    conn.close()

def main():
    delete_redis_key("Picker*")  # Xóa các key Redis trước khi bắt đầu
    # get_picker("mlb", "2024", "ats", db_config,"2024")
    get_picker("mlb", "2024", "ou", db_config,"2024")

if __name__ == "__main__":
    main()


In [ ]:
[{"match_id": "9089", "date_add": "2024-07-22", "status_update": True, "stake": 0, "seq": 3, "result": "pregame", "win_pct": 51.939, "ou_type": "under", "ou_value": 8.5}]